In [9]:
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray, rgb2hsv
from skimage.morphology import disk
from skimage.filters import threshold_otsu
from scipy.ndimage import binary_fill_holes
from skimage.morphology import opening
from skimage.measure import label, regionprops
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [10]:
# Function to calculate color (hue in HSV color space)
def spalva_color(img):
    hsv_img = rgb2hsv(img)
    h_channel = hsv_img[:, :, 0]
    
    gray_img = rgb2gray(img)
    thresh = threshold_otsu(gray_img)
    bw = gray_img > thresh
    
    bw = binary_fill_holes(~bw)
    bw = opening(bw, disk(12))
    
    masked_hue = h_channel[bw]
    hsv_value = np.mean(masked_hue)
    
    return hsv_value


In [11]:
# Function to calculate roundness
def apvalumas_roundness(img):
    gray_img = rgb2gray(img)
    thresh = threshold_otsu(gray_img)
    bw = gray_img > thresh
    
    bw = binary_fill_holes(~bw)
    bw = opening(bw, disk(12))
    
    props = regionprops(label(bw))
    area = props[0].area
    perimeter = props[0].perimeter
    roundness = 4 * np.pi * area / (perimeter ** 2)
    
    return roundness

In [12]:
# Read apple and pear images
apple_images = ['apple_04.jpg','apple_05.jpg', 'apple_06.jpg', 'apple_07.jpg', 'apple_11.jpg', 'apple_12.jpg', 'apple_13.jpg', 'apple_17.jpg', 'apple_19.jpg']
pear_images = ['pear_02.jpg', 'pear_09.jpg', 'pear_01.jpg','pear_03.jpg' ]



In [13]:
# Compute features 
hsv_values_apples = [spalva_color(imread(img)) for img in apple_images]
roundness_apples = [apvalumas_roundness(imread(img)) for img in apple_images]
hsv_values_pears = [spalva_color(imread(img)) for img in pear_images]
roundness_pears = [apvalumas_roundness(imread(img)) for img in pear_images]

In [14]:
# Combine features for training and Create feature matrix 
x1 = np.array(hsv_values_apples + hsv_values_pears)  # HSV (color) values
x2 = np.array(roundness_apples + roundness_pears)    # Roundness values
X = np.column_stack((x1, x2))
y = np.array([1] * len(apple_images) + [-1] * len(pear_images))  


In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [16]:
nb_classifier = GaussianNB()
nb_classifier.fit(X, y)

GaussianNB()

In [17]:
# Predict the labels for the test set and  Evaluate the classifier
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")


Accuracy on the test set: 100.00%
